<a href="https://colab.research.google.com/github/rishurnjan/multilingual-RAG/blob/main/multilingual_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece langchain torch
!pip install faiss-cpu rank_bm25


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 113.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.0 MB/s eta 0:00:00


In [2]:
!pip install pytesseract PyPDF2 imutils
import pytesseract
from PIL import Image
import PyPDF2
import imutils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [3]:
pdf_path="/content/hindi-pdf.pdf"

In [4]:
def detect_pdf_type(pdf_path):

    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        page = pdf_reader.pages[0]
        text = page.extract_text()
        if text:
            return 'text'
        else:
            return 'image'

In [5]:
def extract_text_from_pdf(pdf_path):

    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text



In [6]:
def extract_text_from_image(image_path):

    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

In [7]:
if detect_pdf_type(pdf_path) == 'text':
    pdf_text = extract_text_from_pdf(pdf_path)
else:
    pdf_text = extract_text_from_image(pdf_path)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
)

In [9]:
text_chunks=text_splitter.split_text(pdf_text)


In [10]:
text_chunks[0]

'आ"ट$%फ\'शयल इंटे\'लज/स: एक नया युग आिट र् िफिशयल इंटेिलजेंस (AI) या क ृ ित्रम बुिद्धमत्ता, मानव बुिद्ध के कु छ पहलुओं को क ं प्यूटर प्रणािलयों में अनुकरण करने का प्रयास है। यह क ं प्यूटरों को सीखने, तक र् करने, समस्या हल करने और िनणर्य लेने की क्षमता प्रदान करता है। AI तेजी से हमारे जीवन के िविभन्न क्षेत्रों में प्रवेश कर रहा है, िजससे हमारे काम करने, खेलने और रहने के तरीक े में क्रांितकारी पिरवतर्न हो रहा है। AI के प्रकार AI को मुख्य रूप से दो प्रकारों में वगीर्क ृ त िकया जा सकता है: • तंग AI (Narrow AI): यह एक िविशष्ट कायर् के िलए िडज़ाइन िकया गया AI है, जैसे िक चेहरा पहचानना, शतरंज खेलना या वेबसाइटों का अनुवाद करना। • सामान्य AI (General AI): यह मानव बुिद्ध के समान व्यापक क्षमताओं वाला AI है, जो िकसी भी बौिद्धक कायर् को कर सकता है। AI के अनुप्रयोग AI का उपयोग कई क्षेत्रों में िकया जा रहा'

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def generate_embeddings(text_chunk):
    inputs = tokenizer(text_chunk, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings
chunk_embeddings = [generate_embeddings(chunk) for chunk in text_chunks]
#print("Generated Embeddings for All Chunks:", chunk_embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [12]:
import faiss
import numpy as np

embedding_dim = chunk_embeddings[0].shape[0]
faiss_index = faiss.IndexFlatIP(embedding_dim)

normalized_embeddings = [embedding / np.linalg.norm(embedding) for embedding in chunk_embeddings]
faiss_index.add(np.array(normalized_embeddings).astype('float32'))
print("FAISS index populated with embeddings.")


FAISS index populated with embeddings.


In [13]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')


tokenized_chunks = [word_tokenize(chunk.lower()) for chunk in text_chunks]
bm25 = BM25Okapi(tokenized_chunks)
print("BM25 index created for keyword search.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


BM25 index created for keyword search.


In [14]:
def hybrid_search(query, top_k=5):
    query_embedding = generate_embeddings(query)
    normalized_query_embedding = query_embedding / np.linalg.norm(query_embedding)

    # Vector search with FAISS
    _, faiss_indices = faiss_index.search(np.array([normalized_query_embedding]).astype('float32'), top_k)

    # Keyword search with BM25
    tokenized_query = word_tokenize(query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_indices = np.argsort(bm25_scores)[-top_k:][::-1]

    # Combine FAISS and BM25 results with equal weighting
    combined_indices = list(set(faiss_indices[0]) | set(bm25_indices))
    return combined_indices

query = "AI के प्रकार"
retrieved_indices = hybrid_search(query)
retrieved_chunks = [text_chunks[i] for i in retrieved_indices]
print("Retrieved Chunks:", retrieved_chunks)


Retrieved Chunks: ['आ"ट$%फ\'शयल इंटे\'लज/स: एक नया युग आिट र् िफिशयल इंटेिलजेंस (AI) या क ृ ित्रम बुिद्धमत्ता, मानव बुिद्ध के कु छ पहलुओं को क ं प्यूटर प्रणािलयों में अनुकरण करने का प्रयास है। यह क ं प्यूटरों को सीखने, तक र् करने, समस्या हल करने और िनणर्य लेने की क्षमता प्रदान करता है। AI तेजी से हमारे जीवन के िविभन्न क्षेत्रों में प्रवेश कर रहा है, िजससे हमारे काम करने, खेलने और रहने के तरीक े में क्रांितकारी पिरवतर्न हो रहा है। AI के प्रकार AI को मुख्य रूप से दो प्रकारों में वगीर्क ृ त िकया जा सकता है: • तंग AI (Narrow AI): यह एक िविशष्ट कायर् के िलए िडज़ाइन िकया गया AI है, जैसे िक चेहरा पहचानना, शतरंज खेलना या वेबसाइटों का अनुवाद करना। • सामान्य AI (General AI): यह मानव बुिद्ध के समान व्यापक क्षमताओं वाला AI है, जो िकसी भी बौिद्धक कायर् को कर सकता है। AI के अनुप्रयोग AI का उपयोग कई क्षेत्रों में िकया जा रहा', 'का अनुवाद करना। • सामान्य AI (General AI): यह मानव बुिद्ध के समान व्यापक क्षमताओं वाला AI है, जो िकसी भी बौिद्धक कायर् को कर सकता है। AI के अनुप्रयोग AI का उपयोग कई क्षेत्रों म

In [15]:
retrieved_indices=hybrid_search(query)
retrived_chunks = [text_chunks[i] for i in retrieved_indices]
candidate_embeddings = [normalized_embeddings[i] for i in retrieved_indices]
print("Retrieved Chunks:", retrieved_chunks)

Retrieved Chunks: ['आ"ट$%फ\'शयल इंटे\'लज/स: एक नया युग आिट र् िफिशयल इंटेिलजेंस (AI) या क ृ ित्रम बुिद्धमत्ता, मानव बुिद्ध के कु छ पहलुओं को क ं प्यूटर प्रणािलयों में अनुकरण करने का प्रयास है। यह क ं प्यूटरों को सीखने, तक र् करने, समस्या हल करने और िनणर्य लेने की क्षमता प्रदान करता है। AI तेजी से हमारे जीवन के िविभन्न क्षेत्रों में प्रवेश कर रहा है, िजससे हमारे काम करने, खेलने और रहने के तरीक े में क्रांितकारी पिरवतर्न हो रहा है। AI के प्रकार AI को मुख्य रूप से दो प्रकारों में वगीर्क ृ त िकया जा सकता है: • तंग AI (Narrow AI): यह एक िविशष्ट कायर् के िलए िडज़ाइन िकया गया AI है, जैसे िक चेहरा पहचानना, शतरंज खेलना या वेबसाइटों का अनुवाद करना। • सामान्य AI (General AI): यह मानव बुिद्ध के समान व्यापक क्षमताओं वाला AI है, जो िकसी भी बौिद्धक कायर् को कर सकता है। AI के अनुप्रयोग AI का उपयोग कई क्षेत्रों में िकया जा रहा', 'का अनुवाद करना। • सामान्य AI (General AI): यह मानव बुिद्ध के समान व्यापक क्षमताओं वाला AI है, जो िकसी भी बौिद्धक कायर् को कर सकता है। AI के अनुप्रयोग AI का उपयोग कई क्षेत्रों म

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def rerank_chunks(query, candidate_chunks, candidate_embeddings):
    query_embedding = generate_embeddings(query)

    similarities = cosine_similarity([query_embedding], candidate_embeddings)
    ranked_indices = np.argsort(similarities[0])[::-1]
    ranked_chunks = [candidate_chunks[i] for i in ranked_indices]
    return ranked_chunks
candidate_embeddings = [normalized_embeddings[i] for i in retrieved_indices]
ranked_chunks = rerank_chunks(query, retrieved_chunks, candidate_embeddings)
print("Ranked Chunks:", ranked_chunks)


Ranked Chunks: ['AI के साथ भी कु छ चुनौितयाँ जुड़ी हुई हैं, जैसे िक: • नैितकता: AI का उपयोग नैितक रूप से क ै से िकया जाए, यह एक महत्वपूणर् प्रश्न है। • िनष्पक्षता: AI िसस्टम को िनष्पक्ष और पक्षपाती रिहत बनाना चुनौतीपूणर् हो सकता है। • रोजगार: AI के बढ़ते उपयोग से कु छ नौकिरयों में बदलाव या हािन हो सकती है। िनष्कषर् AI एक तेजी से िवकिसत हो रहा क्षेत्र है, जो हमारे जीवन में कई तरह से प्रभाव डाल रहा है। इसक े लाभों के साथ-साथ चुनौितयों का भी सामना करना होगा। भिवष्य में, AI की प्रगित से और अिधक आश्चयर्जनक िवकास और अनुप्रयोगों की उम्मीद की जा सकती है।', 'AI के साथ भी कु छ चुनौितयाँ जुड़ी हुई हैं, जैसे िक: • नैितकता: AI का उपयोग नैितक रूप से क ै से िकया जाए, यह एक महत्वपूणर् प्रश्न है। • िनष्पक्षता: AI िसस्टम को िनष्पक्ष और पक्षपाती रिहत बनाना चुनौतीपूणर् हो सकता है। • रोजगार: AI के बढ़ते उपयोग से कु छ नौकिरयों में बदलाव या हािन हो सकती है। िनष्कषर् AI एक तेजी से िवकिसत हो रहा क्षेत्र है, जो हमारे जीवन में कई तरह से प्रभाव डाल रहा है। इसक े लाभों के साथ-साथ चुनौितयों का भी सामना करना होगा। भिवष्

In [17]:
import os
import google.generativeai as genai
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] =userdata.get('google_api_key')
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
model = genai.GenerativeModel('gemini-pro')
chat_memory = []
def generate_gemini_response(query, ranked_chunks,chat_memory,memory_limit=512):
    context = " ".join(ranked_chunks)
    memory_text = " ".join(chat_memory[-memory_limit:])
    input_text = (
        f"Conversation history:\n{memory_text}\n\n"
        f"Query: {query}\n"
        f"Context: {context}\n"
        f"Response (Please respond in the language of the provided context):"
    )

    response = model.generate_content(input_text)
    chat_memory.append(f"User: {query}")
    chat_memory.append(f"Model: {response.text}")
    return response.text
response = generate_gemini_response(query, ranked_chunks,chat_memory)
print("Generated Response:", response)
query2 = "सामान्य AI "
response = generate_gemini_response(query2, ranked_chunks, chat_memory)
print("Generated Response:", response)

Generated Response: एआयचे प्रकार एआयला मुख्यत्वे दोन प्रकारात वर्गीकृत केले जाते: • नॅरो एआय (नॅरो एआय): ही एआय विशिष्ट कार्यासाठी डिझाइन केलेली असते, जसे की चेहरा ओळखणे, बुद्धिबळ खेळणे किंवा वेबसाइटचा अनुवाद करणे. • सामान्य एआय (जनरल एआय): ही मानवी बुद्धिमत्तेच्या समान व्यापक क्षमता असणारी एआय असते, जी कोणतेही बौद्धिक कार्य करू शकते. एआयचे अनुप्रयोग एआयचा उपयोग सध्या अनेक क्षेत्रात केला जात आहे, त्यामध्ये समाविष्ट आहेत: • आरोग्य: रोग निदान, औषध शोध आणि वैयक्तिक वैद्यकीय योजना तयार करण्यासाठी. • वित्त: फसवणूक शोधणे, जोखीम मूल्यांकन आणि व्यापारिक निर्णय घेण्यासाठी. • उत्पादन: गुणवत्ता नियंत्रण, स्वयंचलित उत्पादन प्रक्रिया आणि पुरवठा साखळी व्यवस्थापनासाठी. • परिवहन: स्वायत्त वाहने, वाहतूक व्यवस्थापन आणि प्रवासी अनुभव सुधारण्यासाठी. • मनोरंजन: व्हिडिओ गेम्स, सिनेमा आणि संगीत निर्मितीत. एआयसमोरील आव्हाने एआयसोबत, काही आव्हानेही जोडली आहेत, जसे की: • नीतिमत्ता: एआयचा वापर नैतिकरीत्या कसा करावा, हा एक महत्त्वाचा प्रश्न आहे. • निष्पक्षता: एआय प्रणालींना निष्पक्ष आणि भेदभाव रहित करणे आव्हानात्

** Below are some other models that were used for retrieval purpose. **

In [64]:
'''
from transformers import T5Tokenizer, T5ForConditionalGeneration


flan_t5_model_name = "google/flan-t5-base"
flan_t5_tokenizer = T5Tokenizer.from_pretrained(flan_t5_model_name)
flan_t5_model = T5ForConditionalGeneration.from_pretrained(flan_t5_model_name)

def generate_flan_t5_response(query, ranked_chunks):
    context = " ".join(ranked_chunks)
    input_text = f"question: {query} context: {context}"

    input_ids = flan_t5_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True).input_ids

    outputs = flan_t5_model.generate(
        input_ids,
        max_length=150,
        num_beams=7,
        early_stopping=True
    )
    answer = flan_t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer
response = generate_flan_t5_response(query, ranked_chunks)
print("Generated Response:", response)

'''

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Generated Response: 


In [68]:
'''
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_name = "google/mt5-small"
mt5_tokenizer = MT5Tokenizer.from_pretrained(model_name)
mt5_model = MT5ForConditionalGeneration.from_pretrained(model_name)
def generate_mt5_response(query, ranked_chunks):

    context = " ".join(ranked_chunks)
    input_text = f"question: {query} context: {context}"
    input_ids = mt5_tokenizer(input_text, return_tensors="pt", truncation=True,max_length=128 ,padding=True).input_ids
    outputs = mt5_model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True,temperature= 0.7,top_p=0.9)
    answer = mt5_tokenizer.decode(outputs[0], skip_special_tokens=False)
    return answer
response = generate_mt5_response(query, ranked_chunks)
print("Generated Response:", response)
'''

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated Response: <pad> <extra_id_0> है।</s>


In [67]:
'''
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

mbart_model_name = "facebook/mbart-large-50-many-to-many-mmt"
mbart_tokenizer = MBart50TokenizerFast.from_pretrained(mbart_model_name)
mbart_model = MBartForConditionalGeneration.from_pretrained(mbart_model_name)
source_language_code = "en_XX"
target_language_code = "en_XX"

def generate_mbart_response(query, ranked_chunks):

    context = " ".join(ranked_chunks)

    input_text = "explain ai"
    mbart_tokenizer.src_lang = source_language_code
    input_ids = mbart_tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True, padding=True).input_ids
    outputs = mbart_model.generate(
        input_ids,
        max_length=200,
        num_beams=5,
        early_stopping=True
    )
    answer = mbart_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer
response = generate_mbart_response(query, ranked_chunks)
print("Generated Response:", response)
'''

Generated Response: explain ai
